Project-27

In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
Name : Muze Zhao

In [6]:

covid_vaccine <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/covid-vaccination-vs-death_ratio.csv")
covid_vaccine

New names:
• `` -> `...1`
Rows: 32911 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): country, iso_code
dbl  (7): ...1, total_vaccinations, people_vaccinated, people_fully_vaccinat...
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,Afghanistan,AFG,2021-05-11,504502,448878,55624,12,40374668,1.111781
1,Afghanistan,AFG,2021-05-20,547901,470341,77560,10,40374668,1.164941
2,Afghanistan,AFG,2021-05-24,573277,476367,96910,10,40374668,1.179866
3,Afghanistan,AFG,2021-05-26,590454,479372,111082,19,40374668,1.187309
4,Afghanistan,AFG,2021-05-27,593313,479574,113739,14,40374668,1.187809
5,Afghanistan,AFG,2021-05-30,600152,480226,119926,20,40374668,1.189424
6,Afghanistan,AFG,2021-06-02,626290,481690,144600,34,40374668,1.193050
7,Afghanistan,AFG,2021-06-03,630305,481800,148505,27,40374668,1.193323
8,Afghanistan,AFG,2021-06-08,641295,482952,158343,64,40374668,1.196176


In [9]:
covid_vaccine |> 
filter(total_vaccinations < 1000)

...1,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
36,Albania,ALB,2021-02-02,550,549,1,13,2869700,1.913092e-02
2910,Belgium,BEL,2020-12-29,379,378,11,63,11658404,3.242296e-03
2911,Belgium,BEL,2020-12-30,909,907,21,77,11658404,7.779796e-03
2912,Belgium,BEL,2020-12-31,957,955,21,74,11658404,8.191516e-03
2913,Belgium,BEL,2021-01-01,974,972,21,88,11658404,8.337333e-03
2914,Belgium,BEL,2021-01-02,998,996,34,50,11658404,8.543193e-03
9274,Estonia,EST,2020-12-27,191,191,3,2,1324323,1.442246e-02
9275,Estonia,EST,2020-12-28,509,509,9,9,1324323,3.843473e-02
9276,Estonia,EST,2020-12-29,876,876,20,0,1324323,6.614700e-02
